In [20]:
import numpy as np
import pandas as pd
from astropy.io import ascii
from astropy.stats import mad_std

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0 = 0.3)

from astropy.cosmology import WMAP9 as cosmol

In [21]:
south = pd.read_csv('/Users/rosaliaobrien/research/GALFIT_CLEAR/running_GALFIT/South/output_data_all.csv')
north = pd.read_csv('/Users/rosaliaobrien/research/GALFIT_CLEAR/running_GALFIT/North/output_data_all.csv')
south['Field'] = ['South']*len(south['gid'])
north['Field'] = ['North']*len(north['gid'])
df = pd.concat([north,south],ignore_index=True, sort=False)

# badfit_df = pd.read_csv('/Users/rosaliaobrien/research/GALFIT_CLEAR/running_GALFIT/dropped_fit.csv', header=None)
# badfit = badfit_df[0].tolist()

# for index, gid in zip(df.index, df['gid']):
#     if gid in badfit:
#         df = df.drop([index])

df['850-160_arcsec'] = df['850-160']*1.6666667e-5*3600
df['125-160_arcsec'] = df['125-160']*1.6666667e-5*3600

estrada2020 = ascii.read('/Users/rosaliaobrien/research/GALFIT_CLEAR/running_GALFIT/apjaba004t2_mrt.txt').to_pandas()
estrada2020[['field','gid']] = estrada2020['ID'].str.split('-',expand=True)
estrada2020['gid'] = estrada2020['gid'].astype(int)
df = df.merge(estrada2020, on='gid', how='inner')

df['t50'] = cosmo.lookback_time(df['z50'])-cosmo.lookback_time(df['zgrism'])

df['z50_error'] = (df['E_z50']+df['e_z50'])/2
df['zgrism_error'] = (df['E_zgrism']+df['e_zgrism'])/2

lowert50 = np.array(cosmo.lookback_time(df['e_z50']) - cosmo.lookback_time(df['zgrism']))
uppert50 = np.array(cosmo.lookback_time(df['E_z50']) - cosmo.lookback_time(df['zgrism']))
e_t50 = uppert50 - df['t50']
E_t50 = df['t50'] - lowert50
df['t50_error'] = (e_t50+E_t50)/2

In [22]:
df['kpc_proper_per_arcmin'] = cosmol.kpc_proper_per_arcmin(df['zgrism'])

In [23]:
df['radius850_arcsec'] = df['radius_850']*1.6666667e-5*3600
df['radius850_kpc'] = df['radius850_arcsec']/60*df['kpc_proper_per_arcmin']

df['radius125_arcsec'] = df['radius_125']*1.6666667e-5*3600
df['radius125_kpc'] = df['radius125_arcsec']/60*df['kpc_proper_per_arcmin']

df['radius160_arcsec'] = df['radius_160']*1.6666667e-5*3600
df['radius160_kpc'] = df['radius160_arcsec']/60*df['kpc_proper_per_arcmin']

In [24]:
df['850-160_kpc'] = df['850-160_arcsec']/60*df['kpc_proper_per_arcmin']
df['125-160_kpc'] = df['125-160_arcsec']/60*df['kpc_proper_per_arcmin']

In [25]:
for gid in df['gid']:
    print(gid)

29879
27006
22358
36838
37186
13774
32108
23459
24795
14158
29183
24177
23081
22213
33453
23758
22246
26673
27951
37340
12793
36161
22363
27185
16758
12078
22633
37955
37210
30358
23857
21724
37325
22027
34694
38102
28451
20432
17746
34419
13191
14713
17070
21156
37686
36530
24345
16574
21427
33775
33780
17735
19850
14132
21738
32933
17599
23018
41147
47140
46001
38191
39850
19148
42221
39170
43615
39241
39631
45972
44620
29928
47691
39805
40476
37828
40597
38785
35774
40862
46066
39804
45775
40623
40223
39649
42487
38843
39012
41520
44042
42615
41148
24569
24315
43572
44133
48464


In [26]:
df = df.drop_duplicates()

gids = []
for gid in df['gid']:
    if gid in gids:
        print(gid)
    gids.append(gid)

In [27]:
df.sort_values(by='850-160_kpc')

,gid,mag_160,radius_160,sersic_index_160,axis_ratio_160,PA_160,circ_radius_160,mag_125,radius_125,sersic_index_125,...,t50_error,kpc_proper_per_arcmin,radius850_arcsec,radius850_kpc,radius125_arcsec,radius125_kpc,radius160_arcsec,radius160_kpc,850-160_kpc,125-160_kpc
72,47691,21.5364,9.8864,4.4360,0.8575,-64.8987,9.154935,21.0766,8.7920,2.7959,...,-2.763129,501.724638,0.173508,1.450887,0.527520,4.411163,0.593184,4.960251,-3.390507,-0.861454
70,44620,22.6035,6.3238,5.4539,0.7054,-82.2125,5.311239,22.0526,5.0516,3.6403,...,-0.126147,497.883882,0.000000,0.000000,0.303096,2.515110,0.379428,3.148518,-2.644380,-0.642378
64,42221,21.9439,12.3911,8.4311,0.7237,25.1428,10.541178,21.6707,5.8122,3.4469,...,-0.431781,490.988401,0.403980,3.305825,0.348732,2.853723,0.743466,6.083886,-2.436745,-2.973602
78,35774,21.2502,11.6329,6.3533,0.8987,-16.6454,11.027965,21.0219,5.4301,3.2506,...,0.000000,510.340821,0.552594,4.700188,0.325806,2.771202,0.697974,5.936744,-1.130552,-2.984317
17,26673,21.7467,7.8746,9.9124,0.6266,87.2619,6.233381,21.1624,6.7628,8.2509,...,-0.565340,481.715158,0.261930,2.102928,0.405768,3.257743,0.472476,3.793314,-1.088126,-0.334147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,23758,20.8273,14.0107,7.6652,0.8247,-86.7208,12.723536,20.1136,17.3690,8.1489,...,-0.713912,481.576787,1.681812,13.498694,1.042140,8.364507,0.840642,6.747228,6.023714,1.414625
11,24177,21.2602,10.3221,6.6387,0.7744,15.0727,9.083448,20.3937,15.8577,8.0000,...,-0.893459,481.019639,1.461144,11.713983,0.951462,7.627865,0.619326,4.965133,6.499467,2.520719
50,33780,22.1309,10.7768,5.2589,0.9783,-50.8161,10.659230,21.7214,13.0909,4.7876,...,-2.829781,516.025094,1.606182,13.813837,0.785454,6.755233,0.646608,5.561099,6.901288,1.130718
45,36530,21.8742,13.4255,9.7958,0.8329,36.9528,12.252562,21.2044,16.2096,9.4889,...,-1.058506,514.802020,5.650350,48.480194,0.972576,8.344735,0.805530,6.911475,36.431985,1.062227


In [28]:
df['unc_850-160_kpc'] = mad_std(df['850-160_kpc'])/np.sqrt(len(df['850-160_kpc']))
df['unc_125-160_kpc'] = mad_std(df['125-160_kpc'])/np.sqrt(len(df['125-160_kpc']))

In [29]:
df.to_csv('/Users/rosaliaobrien/research/GALFIT_CLEAR/running_GALFIT/output_data.csv', index=False)